<a href="https://colab.research.google.com/github/hanzlikhan/AI-Smart-Agriculture/blob/main/langchain_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG:
- llm trained in a limited time
- to give upto date answer or questions for correct reponse that's RAG

- Vector databasek

tensorflow website to check embeddings

- rag setup
1. vector database  -> pinecone
2. pipeline    
- user query and other things

In [1]:
pip install -qU langchain-pinecone langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0

In [2]:
from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [4]:
index_name = "online-rag-project"

pc.create_index(
    name=index_name, # like database
    dimension=768, # through google
    metric= "cosine", # angle
    spec=ServerlessSpec(cloud="aws", region="us-east-1") # region and aws at which it formed
)

index = pc.Index(index_name)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'x-cloud-trace-context': '5e18746a3a25c42d07acb4a604faecb1', 'date': 'Tue, 25 Mar 2025 17:33:59 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [5]:
import time
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec

# Re-initialize 'pc' here by fetching the API key and creating a Pinecone instance
pinecone_api_key = userdata.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

index_name = "online-rag-project"  # change if desired

pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

index = pc.Index(index_name)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'x-cloud-trace-context': '312ebe96ec2f61fe28c3e3ba85f41b27', 'date': 'Tue, 25 Mar 2025 17:34:17 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
vector = embeddings.embed_query("we are building a RAG text!")
vector[:5]

[-0.050278227776288986,
 -0.008960901759564877,
 -0.053891099989414215,
 -0.012207981199026108,
 -0.02719450742006302]

- semantic
 - similarities
 - word to get similarities, we look at the nearest words
- cosine search
   - find any word in angle in all directions
- vector is in just like it store data in their

- embeddings -> giving different value to different words

In [6]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [8]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['66999736-f251-47fa-a0a8-4287842b9811',
 '83c485f3-8581-4683-b7a6-3ddf4bfd4cd9',
 'bd4a78c9-ca3d-4b94-9857-e149a0c38ec8',
 '71e3ff74-6e1f-4926-82a9-ad44abff663b',
 '120dc904-18b5-443c-ba62-5d5f51500e4b',
 '938deb81-a66b-49e7-8b17-ab6830cd4942',
 '78c3e86b-aade-42a7-8744-368b07d54eed',
 '33119f22-cbd7-403b-8b4e-702095bce417',
 '727732a5-65e0-4350-8f89-3b523c8395f2',
 '4aa7222a-f2c5-4aa4-80cc-94303cece3b0']

# Data retrieve

In [9]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [10]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.639507] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


# implementation of rag

In [ ]:
!pip install  --quiet --upgrade langchain-text-splittlers langchain-community langchain-google-genai

!pip install -qU pypdf

import os
from google.colab import userdata
userdata.get("GOOGLE_API_KEY")
os.environ['GOOGLE_API_KEY'] = userdata.get("GOOGLE_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash")

from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
embeddings.embed_query("we are building a RAG text!")

from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

index_name = "quickstart4"
pc = Pinecone(api_key= 'pcsk_3FfEVW_SMaEpCtQgpYXAGNnVHGxLrhCvG7XzgpNeG6WuWoKQGV75WfYLSYc21dB5ooBsxy')

pc.create_index(
    name = index_name,
    dimension=768,
    metric = "cosine",
    spec = ServerlessSpec(
        cloud="aws",
        region="us-east-1")
)

index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

from langchain.document_loaders import PyPDFLoader

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# load document

loader = TextLoader("documents.txt")
documents = loader.load()

# split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)